<a href="https://colab.research.google.com/github/qetibakh/ML-Final/blob/main/model_experiment_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install lightgbm

In [13]:
import wandb
from google.colab import userdata # If you are using Colab and Colab Secrets

try:
    # If you stored your API key in Colab Secrets
    wandb_api_key = userdata.get('WANDB_API_KEY')
    if wandb_api_key:
        wandb.login(key=wandb_api_key)
        print("Wandb login successful!")
    else:
        # If not using Colab Secrets, you might need to log in interactively
        # or set the WANDB_API_KEY environment variable
        print("Wandb API key not found in Colab Secrets. Attempting interactive login or using environment variable.")
        wandb.login()

    # Initialize a new Wandb run for this notebook/experiment
    # You can give it a different name if you like, e.g., 'lightgbm-training-notebook'
    run = wandb.init(project='walmart-sales-forecasting', name='lightgbm-model-training')
    print(f"Wandb run initialized: {run.name}")

except Exception as e:
    print(f"Wandb initialization or login failed: {e}")

# Now you can safely use wandb.config.update(), wandb.log(), etc.
# Remember to call run.finish() at the end of your notebook

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: qetibaxtura (qetibaxtura-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Wandb login successful!


Wandb run initialized: lightgbm-model-training


In [5]:
import pandas as pd
import os
from google.colab import drive

# Mount Google Drive to access the saved files
drive.mount('/content/drive')

# Define the path where you saved the processed data
save_path = '/content/drive/MyDrive/Colab Notebooks/walmart_sales_data/processed_data' # Make sure this matches the path you used for saving

# Load the processed dataframes
train_data = pd.read_csv(os.path.join(save_path, 'train_data_processed.csv'))
val_data = pd.read_csv(os.path.join(save_path, 'val_data_processed.csv'))
test_merged_df = pd.read_csv(os.path.join(save_path, 'test_data_processed.csv'))

print("Processed data loaded successfully.")

# Display the first few rows of the loaded dataframes to verify
print("Train data head:")
display(train_data.head())

print("\nValidation data head:")
display(val_data.head())

print("\nTest data head:")
display(test_merged_df.head())

Mounted at /content/drive
Processed data loaded successfully.
Train data head:


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Days_To_Thanksgiving,Days_To_Christmas,IsThanksgivingWeek,IsChristmasWeek,MarkDown1_IsHoliday,Week_OfYear_Cat,Quarter,Store_Type_A,Store_Type_B,Store_Type_C
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,293,323,False,False,0.0,5,1,True,False,False
1,1,2,2010-02-05,50605.27,False,42.31,2.572,0.0,0.0,0.0,...,293,323,False,False,0.0,5,1,True,False,False
2,1,3,2010-02-05,13740.12,False,42.31,2.572,0.0,0.0,0.0,...,293,323,False,False,0.0,5,1,True,False,False
3,1,4,2010-02-05,39954.04,False,42.31,2.572,0.0,0.0,0.0,...,293,323,False,False,0.0,5,1,True,False,False
4,1,5,2010-02-05,32229.38,False,42.31,2.572,0.0,0.0,0.0,...,293,323,False,False,0.0,5,1,True,False,False



Validation data head:


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Days_To_Thanksgiving,Days_To_Christmas,IsThanksgivingWeek,IsChristmasWeek,MarkDown1_IsHoliday,Week_OfYear_Cat,Quarter,Store_Type_A,Store_Type_B,Store_Type_C
0,1,1,2012-07-06,17823.37,False,81.57,3.227,12218.76,94.4,192.83,...,139,172,False,False,0.0,27,3,True,False,False
1,1,2,2012-07-06,49771.92,False,81.57,3.227,12218.76,94.4,192.83,...,139,172,False,False,0.0,27,3,True,False,False
2,1,3,2012-07-06,8473.71,False,81.57,3.227,12218.76,94.4,192.83,...,139,172,False,False,0.0,27,3,True,False,False
3,1,4,2012-07-06,44936.96,False,81.57,3.227,12218.76,94.4,192.83,...,139,172,False,False,0.0,27,3,True,False,False
4,1,5,2012-07-06,19684.48,False,81.57,3.227,12218.76,94.4,192.83,...,139,172,False,False,0.0,27,3,True,False,False



Test data head:


,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,Days_To_Thanksgiving,Days_To_Christmas,IsThanksgivingWeek,IsChristmasWeek,MarkDown1_IsHoliday,Week_OfYear_Cat,Quarter,Store_Type_A,Store_Type_B,Store_Type_C
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,20,53,False,False,0.00,44,4,True,False,False
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,13,46,False,False,0.00,45,4,True,False,False
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,6,39,True,False,0.00,46,4,True,False,False
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,1,32,True,False,883.59,47,4,True,False,False
4,1,1,2012-11-30,False,52.34,3.207,2460.03,0.00,3838.35,150.57,...,8,25,False,False,0.00,48,4,True,False,False


Now that we have a baseline LightGBM model trained and evaluated, we can proceed with logging this experiment using MLflow or Wandb as outlined in the plan. We can also explore hyperparameter tuning or feature engineering to improve the model's performance.

In [14]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import wandb # Import wandb

# Assuming train_data and val_data DataFrames are already available from previous steps

# Define features (X) and target variable (y)
# Exclude non-feature columns like 'Store', 'Date', 'Weekly_Sales', and any other columns not used for training
features_to_exclude = ['Store', 'Date', 'Weekly_Sales']
features = [col for col in train_data.columns if col not in features_to_exclude]
target = 'Weekly_Sales'

X_train = train_data[features].copy() # Use .copy() to avoid SettingWithCopyWarning
y_train = train_data[target].copy()
X_val = val_data[features].copy()
y_val = val_data[target].copy()


# Handle any remaining NaNs in the feature sets, for example by filling with 0 or a suitable imputation strategy
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)

# Select numeric and boolean columns as features. Explicitly drop any remaining object type columns.
X_train = X_train.select_dtypes(include=[np.number, bool])
X_val = X_val.select_dtypes(include=[np.number, bool])

# Check for and drop any remaining 'object' type columns just in case
object_cols_train = X_train.select_dtypes(include='object').columns
if len(object_cols_train) > 0:
    print(f"Dropping object columns from X_train: {list(object_cols_train)}")
    X_train = X_train.drop(columns=object_cols_train)

object_cols_val = X_val.select_dtypes(include='object').columns
if len(object_cols_val) > 0:
    print(f"Dropping object columns from X_val: {list(object_cols_val)}")
    X_val = X_val.drop(columns=object_cols_val)


# Display data types for debugging
print("\nX_train dtypes before model fit:")
print(X_train.dtypes)
print("\nX_val dtypes before model fit:")
print(X_val.dtypes)

# Convert validation data to NumPy arrays for eval_set (temporarily not used in fit below)
X_val_np = X_val.values
y_val_np = y_val.values

# Convert training data to NumPy arrays
X_train_np = X_train.values
y_train_np = y_train.values


# Initialize and train the LightGBM Regressor
lgb_params = {
    'objective': 'regression_l1', # MAE objective
    'metric': 'mae',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}

# Log hyperparameters to Wandb
# Ensure wandb.init() has been called in a previous cell
wandb.config.update(lgb_params)


model = lgb.LGBMRegressor(**lgb_params)
# Temporarily remove eval_set and callbacks for diagnosis
model.fit(X_train_np, y_train_np)
#          eval_set=[(X_val_np, y_val_np)], # Use NumPy arrays for eval_set
#          eval_metric='mae',
#          callbacks=[lgb.early_stopping(100, verbose=False)])


# Make predictions on the validation set
predictions = model.predict(X_val)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_val, predictions))
mae = mean_absolute_error(y_val, predictions)

# Log metrics to Wandb
wandb.log({"val_rmse": rmse, "val_mae": mae})

print(f"LightGBM Validation RMSE: {rmse}")
print(f"LightGBM Validation MAE: {mae}")

# Remember to call wandb.finish() at the end of your experiment run.
# This is often done in a separate cell or after all your logging is complete for a run.
# For a single script, you might uncomment the line below:
# if wandb.run:
#     wandb.finish()


X_train dtypes before model fit:
Dept                           int64
IsHoliday                       bool
Temperature                  float64
Fuel_Price                   float64
MarkDown1                    float64
MarkDown2                    float64
MarkDown3                    float64
MarkDown4                    float64
MarkDown5                    float64
CPI                          float64
Unemployment                 float64
Size                           int64
Year                           int64
Month                          int64
Week                           int64
Day                            int64
DayOfWeek                      int64
Weekly_Sales_Lag1            float64
Weekly_Sales_RollingMean4    float64
Days_To_Thanksgiving           int64
Days_To_Christmas              int64
IsThanksgivingWeek              bool
IsChristmasWeek                 bool
MarkDown1_IsHoliday          float64
Week_OfYear_Cat                int64
Quarter                        int64
Stor

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LightGBM Validation RMSE: 4141.377555114245
LightGBM Validation MAE: 2192.1305223522922


https://github.com/enelene/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/Qeto/LGBM_prediction_plot.png